In [ ]:
import os

from pathlib import Path

import matplotlib.pyplot as plt

from lhotse.recipes.ami import download, prepare_ami
from lhotse.features import FeatureSetBuilder, Spectrogram
from lhotse.cut import CutSet
from lhotse.dataset.vad import VadDataset

# Settings for paths

In [ ]:
root_dir = Path('data')
output_dir = root_dir / 'ami_nb'

# Download and untar

In [ ]:
download_and_untar(root_dir)

# Prepare audio and supervision manifests

In [ ]:
ami_manifests = prepare_ami(root_dir, output_dir, asr_task=False)

# Extract features

In [ ]:
example = ami_manifests['dev']

feature_set_builder = FeatureSetBuilder(
    feature_extractor=Spectrogram(),
    output_dir=f'{output_dir}/feats_example'
)
feature_set = feature_set_builder.process_and_store_recordings(
    recordings=example['audio'],
    num_jobs=os.cpu_count()
)
example['feats'] = feature_set

In [ ]:
cut_set = CutSet.from_manifests(supervision_set=example['supervisions'], feature_set=feature_set)
example['cuts'] = cut_set

# Make pytorch Dataset

In [ ]:
vad_dataset = VadDataset(example['cuts'])

# Illustation of two examples: voice / unvoice

In [ ]:
sample = vad_dataset[0]
print('Voice' if sample['is_voice'] else 'Unvoice')
plt.matshow(sample['features'].transpose(0, 1).flip(0))

In [ ]:
sample = vad_dataset[1]
print('Voice' if sample['is_voice'] else 'Unvoice')
plt.matshow(sample['features'].transpose(0, 1).flip(0))